In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\ali\Desktop\projects\medical chatbot\endtoendmedicalchatbot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logger.info("hello")

In [2]:
# def load_pdf_file(data):
        
#     loader = DirectoryLoader(data,glob="*.pdf",
#                   loader_cls=PyPDFLoader)
#     documents=loader.load()
#     return documents
    



In [3]:
# extracted_data = load_pdf_file(data="../Data/")

In [4]:
# extracted_data

In [5]:
# def text_split(extracted_data):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
#     text_chunks = text_splitter.split_documents(extracted_data)
#     return text_chunks

In [6]:
# text_chunks = text_split(extracted_data)
# print("length of chunks",len(text_chunks))

In [7]:
# text_chunks[2]

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings


In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
query_result = embeddings.embed_query("hello")
print(len(query_result))

384


In [11]:
from dotenv import load_dotenv
import os
load_dotenv()

api =  os.environ["PINECONE_API_KEY"]

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key=api)
index_name = "medicalbot"
# pc.create_index(
#     name = index_name,
#     dimension=384,
#     metric="cosine",
#    spec= ServerlessSpec(cloud="aws",
#                          region="us-east-1") 
# )

In [15]:
# Storing
from langchain_pinecone import Pinecone
# Pinecone.from_documents(documents=text_chunks,embedding=embeddings,index_name = index_name)

In [16]:
# Loading 
docsearch = Pinecone.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [17]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [47]:
retrived_docs = retriever.invoke("What is Amniotic sac ?")

In [19]:
print("\n".join([retrived_docs[0].page_content for i in [0,1,2]]))

Overcoming Sexual Dysfunction and Reclaiming Your Sex
Life. Henry Holt & Company, Inc., 2001.
Reichman, Judith. I’m Not in the Mood: What Every Woman
Should Know About Improving Her Libido.Quill Publish-
ing, 1999.
Rako, Susan M.D. The Hormone of Desire: The Truth About
Testosterone, Sexuality, and Menopause.Three Rivers
Press, 1999.
PERIODICALS
“Restoring Sexual Health.” Consumer Reports On Health.
(March 2001): 8-10.
“Consumer Update: Female Sexual Problems.” American Asso-
Overcoming Sexual Dysfunction and Reclaiming Your Sex
Life. Henry Holt & Company, Inc., 2001.
Reichman, Judith. I’m Not in the Mood: What Every Woman
Should Know About Improving Her Libido.Quill Publish-
ing, 1999.
Rako, Susan M.D. The Hormone of Desire: The Truth About
Testosterone, Sexuality, and Menopause.Three Rivers
Press, 1999.
PERIODICALS
“Restoring Sexual Health.” Consumer Reports On Health.
(March 2001): 8-10.
“Consumer Update: Female Sexual Problems.” American Asso-
Overcoming Sexual Dysfunction and Recl

In [30]:
from groq import Groq
from dotenv import load_dotenv
import os

# Load variables from .env
load_dotenv()

# Now the API key is available
client = Groq(api_key=os.environ["GROQ_API_KEY"])

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Answer everythin in one sentence, or two sentence "},
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama-3.3-70b-versatile",
    max_tokens=40,
)

print(chat_completion.choices[0].message.content)


Fast language models are crucial in natural language processing as they enable efficient and accurate text analysis, generation, and understanding, allowing for real-time applications such as language translation, sentiment analysis, and chatbots.


In [35]:
import os
from groq import Groq
from langchain_core.language_models.llms import LLM

class GroqLLM(LLM):
    model_name: str = "llama-3.3-70b-versatile"

    def _call(self, prompt: str, stop=None, **kwargs) -> str:
        client = Groq(api_key=os.environ["GROQ_API_KEY"])
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model_name
        )
        return response.choices[0].message.content

    @property
    def _identifying_params(self):
        return {"model_name": self.model_name}

    @property
    def _llm_type(self):
        return "groq"


In [36]:
llm = GroqLLM()

In [ ]:
from groq import Groq
import os

# 1) Custom Groq LLM
from langchain_core.language_models.llms import LLM
class GroqLLM(LLM):
    model_name: str = "llama-3.3-70b-versatile"
    def _call(self, prompt: str, stop=None, **kwargs) -> str:
        client = Groq(api_key=os.environ["GROQ_API_KEY"])
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model_name,
        )
        return response.choices[0].message.content

    @property
    def _identifying_params(self):
        return {"model_name": self.model_name}
    @property
    def _llm_type(self):
        return "groq"

llm = GroqLLM()

# 2) Do your own RAG manually
# retriever is your vectorstore retriever, e.g., Pinecone or FAISS
query = "What is Amniotic sac"
docs = retrived_docs # returns list[Document]

# 3) Build context string
context = "\n\n".join(d.page_content for d in docs)

prompt = f"""
Use the following context to answer the question.

Context:
{context}

Question:
{query}
If question is out of context , just tell i don't know
"""

answer = llm.invoke(prompt)
print(answer)


According to the context, the Amniotic sac is "The membranous sac that surrounds the embryo and fills with watery fluid as pregnancy advances."


In [63]:
def ask(query):
    llm = GroqLLM()
    query = query
    docs = retriever.invoke(query)

    # 3) Build context string
    context = "\n\n".join(d.page_content for d in docs)

    prompt = f"""
    Use the following context to answer the question.

    Context:
    {context}

    Question:
    {query}
    If question is out of context , just tell i don't know
    """
    # print(context)
    answer = llm.invoke(prompt)
    return answer


In [64]:
print(ask("what is Cryotherapy "))

Cryotherapy is a technique that uses an extremely cold liquid or instrument to freeze and destroy abnormal skin cells that require removal. It is also called cryosurgery.
